In [17]:
import pandas as pd
import fantasy_conversion
import predicted_fantasy
import predictive_analysis
import team_performance
from dash import Dash, html, dcc, callback, Output, Input, State
import pandas as pd
import plotly.express as px

In [18]:
nba_api_df = pd.read_csv('pic16b_nba_stats.csv')
sorted_fantasy_df = fantasy_conversion.fantasy_stats(nba_api_df)

In [3]:
sorted_fantasy_df

,Year,PLAYER,TEAM,Pos,G,FGM,FG3M,FTM,REB,AST,STL,BLK,TOV,Fantasy Points
4144,2023-24,Wesley Matthews,ATL,SG,36.0,21.6,75.6,18.0,64.80,32.4,28.8,21.6,-7.2,255.60
2979,2022-23,De'Andre Hunter,ATL,SF,67.0,562.8,301.5,174.2,337.68,140.7,67.0,40.2,-80.4,1543.68
1867,2020-21,Onyeka Okongwu,ATL,C,50.0,190.0,0.0,35.0,198.00,30.0,50.0,70.0,-30.0,543.00
3404,2022-23,Aaron Holiday,ATL,PG,63.0,113.4,113.4,25.2,90.72,132.3,75.6,25.2,-37.8,538.02
562,2018-19,Justin Anderson,ATL,SF,48.0,76.8,72.0,24.0,103.68,36.0,48.0,28.8,-24.0,365.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,2021-22,Kyle Kuzma,WAS,PF,66.0,594.0,376.2,158.4,673.20,346.5,79.2,118.8,-171.6,2174.70
3524,2022-23,Isaiah Todd,WAS,PF,6.0,3.6,3.6,1.8,14.40,6.3,2.4,0.0,-1.8,30.30
3523,2022-23,Jordan Schakel,WAS,SF,2.0,0.0,3.0,0.0,0.00,1.5,2.0,0.0,-0.0,6.50
4085,2023-24,Jules Bernard,WAS,SG,19.0,34.2,34.2,5.7,31.92,22.8,7.6,3.8,-7.6,132.62


In [5]:
next_year_points = predicted_fantasy.predict_next_fantasy_point(sorted_fantasy_df)

In [6]:
next_year_points

,PLAYER,Pred Fantasy Points,Year
0,Luka Dončić,4019.147748,2024-25
1,Nikola Jokić,3844.610878,2024-25
2,Shai Gilgeous-Alexander,3802.694449,2024-25
3,Giannis Antetokounmpo,3706.630366,2024-25
4,Jayson Tatum,3541.779721,2024-25
...,...,...,...
622,Paul Millsap,-46.231110,2024-25
623,Aleksej Pokusevski,-53.320579,2024-25
624,Danny Green,-63.951514,2024-25
625,Théo Maledon,-75.373209,2024-25


In [3]:
print(team_performance.team_analysis(sorted_fantasy_df, 'WAS'))

{'Total Players': 24, 'Average Fantasy Points': 741.0574999999999, 'Total Fantasy Points': 17785.379999999997}


In [4]:
team_colors = {
    'ATL': '#E03A3E',  # Atlanta Hawks
    'BOS': '#007A33',  # Boston Celtics
    'BLK': '#000000',  # Brooklyn Nets
    'CHA': '#1D1160',  # Charlotte Hornets
    'CHI': '#CE1141',  # Chicago Bulls
    'CLE': '#6A1E1C',  # Cleveland Cavaliers
    'DAL': '#0061F2',  # Dallas Mavericks
    'DEN': '#0E76A8',  # Denver Nuggets
    'DET': '#006BB6',  # Detroit Pistons
    'GSW': '#006BB6',  # Golden State Warriors
    'HOU': '#CE1141',  # Houston Rockets
    'IND': '#FDBB30',  # Indiana Pacers
    'LAC': '#1D428A',  # Los Angeles Clippers
    'LAL': '#552583',  # Los Angeles Lakers
    'MEM': '#5D2F6E',  # Memphis Grizzlies
    'MIA': '#98002E',  # Miami Heat
    'MIL': '#00471B',  # Milwaukee Bucks
    'MIN': '#236192',  # Minnesota Timberwolves
    'NOP': '#002B5C',  # New Orleans Pelicans
    'NYK': '#006BB6',  # New York Knicks
    'OKC': '#007A33',  # Oklahoma City Thunder
    'ORL': '#0077C0',  # Orlando Magic
    'PHI': '#006BB6',  # Philadelphia 76ers
    'PHO': '#E56020',  # Phoenix Suns
    'POR': '#E03A3E',  # Portland Trail Blazers
    'SAC': '#7A4BB1',  # Sacramento Kings
    'SAS': '#B5B5B5',  # San Antonio Spurs
    'TOR': '#CE1141',  # Toronto Raptors
    'UTH': '#007A33',  # Utah Jazz
    'WAS': '#002F6C'   # Washington Wizards
}

In [23]:

#setting up our data 
filtered_df = sorted_fantasy_df.sort_values(by='Year')
df = filtered_df

#initialize the website
app = Dash()

#create the layout
app.layout = html.Div([
    html.H1("NBA Player Stats Dashboard", style={'text-align': 'center'}),
    html.Hr(),
    dcc.Dropdown(
        id='stat-dropdown',
        options=[
            {'label': col, 'value': col} for col in df.columns[4:]  # List options from FGM to Fantasy Points
        ],
        value='Fantasy Points',  #Default 
        style={'width': '50%'}
    ),
    html.Br(),
    dcc.RadioItems(
        options=['Single Player', 'Compare Players'],
        value='Single Player',
        id='mode-selector'
    ),
    html.Br(),
    html.Div([
        html.Label("Player Name:"),
        dcc.Input(id='player-name-input', value='DeMar DeRozan', type='text'),
        html.Label("Second Player Name:"),
        dcc.Input(id='second-player-name-input', value = 'Luka Dončić', type='text')
    ]),
    html.Br(),
    dcc.Graph(figure={}, id='player-fantasy-graph'), 
    html.Br(),
     html.Div([
        dcc.Markdown(
            """
            **Disclaimer:**  
            Fantalyzer is for informational and entertainment purposes only. 
            We are not responsible for any financial losses or decisions made based on the information provided. 
            Fantalyzer does not endorse or encourage gambling in any form.
            """
        ),
        html.Br(),
        html.P("© 2024 NBA Fantalyzer", style={'fontSize': '12px', 'textAlign': 'center'})
    ], style={'marginTop': '50px', 'padding': '10px', 'borderTop': '1px solid #ccc', 'fontSize': '14px', 'color': '#666'}),
    html.Br(),
    html.H2("Fantasy Trade Simulator", style={'text-align': 'center'}),
    html.Hr(),
    html.Br(),
    html.Div([
        html.Div([
            #Selecting team one
            html.Div([
                html.Label("Select Team 1"),
                dcc.Dropdown(
                    id='team1-dropdown',
                    options=[{'label': team, 'value': team} for team in df['TEAM'].unique()],
                    value='DAL',  
                    style={'display': 'inline-block', 'width': '100%'}
                )
            ], style={'width': '100%', 'display': 'inline-block'}), 
            html.Br(),
            #Selecting the players from team1
            html.Div([
                html.Label("Select Players from Team 1"),
                dcc.Checklist(
                    id='team1-players',
                    options=[],  
                    style={'width': '100%'}
                )
            ], style={'width': '100%', 'display': 'inline-block'})  

        ], style={'width': '48%', 'display': 'inline-block'}),  

        #Team2
        html.Div([
            #Dropdown
            html.Div([
                html.Label("Select Team 2"),
                dcc.Dropdown(
                    id='team2-dropdown',
                    options=[{'label': team, 'value': team} for team in df['TEAM'].unique()],
                    value='BOS',  
                    style={'display': 'inline-block', 'width': '100%'}
                )
            ], style={'width': '100%', 'display': 'inline-block'}),
            html.Br(),
            #Selecting team2
            html.Div([
                html.Label("Select Players from Team 2"),
                dcc.Checklist(
                    id='team2-players',
                    options=[],  
                    style={'width': '100%'}
                )
            ], style={'width': '100%', 'display': 'inline-block'}) 

        ], style={'width': '48%', 'display': 'inline-block'}),  

    ], style={'display': 'flex', 'justify-content': 'space-between', 'gap': '10px'}),  

    
    html.Br(),
    # Button to simulate trade
    html.Button("Simulate Prospective Trade", id='trade-button', style={'font-size':'20px',
                                                                        'padding':'15px 30px',
                                                                        'margin':'20px auto',
                                                                        'background-color':'#007BFF',
                                                                        'color':'white',
                                                                        'border-radius': '5px'}
    ),
    # Display team analysis
    html.H3("Original Team Analyis"),
    html.Div(id='team-analysis'),
    html.Div(id='trade-analysis')

])



# Callback to update the graph based on the selected stat and player inputs
@app.callback(
    Output('player-fantasy-graph', 'figure'),
    [Input('stat-dropdown', 'value'),
     Input('mode-selector', 'value'),
     Input('player-name-input', 'value'),
     Input('second-player-name-input', 'value')]
)

def update_graph(stat, mode, player_name, second_player_name):
    fig = px.bar(title="Enter player names to see their fantasy stats")
    
    #Single Player 
    if mode == 'Single Player' and player_name:
        filtered_df = df[df['PLAYER'].str.contains(player_name, case=False, na=False)]
        fig = px.bar(
            filtered_df,
            x='Year',
            y=stat,
            color='TEAM',
            title=f"{player_name}'s {stat} Over the Years",
            color_discrete_map=team_colors
        )
    
    #Compare Players
    elif mode == 'Compare Players' and player_name and second_player_name:
        filtered_df = df[df['PLAYER'].str.contains(player_name + '|' + second_player_name, case=False, na=False)]
        fig = px.bar(
            filtered_df,
            x='Year',
            y=stat,
            color='TEAM',
            barmode='group',
            title=f"Comparison of {player_name} and {second_player_name} {stat}", 
            color_discrete_map=team_colors
        )

    return fig

@app.callback(
    [Output('team1-players', 'options'),
     Output('team2-players', 'options'),
    Output('team-analysis', 'children')],
    [Input('team1-dropdown', 'value'),
     Input('team2-dropdown', 'value')]
)
def update_players(team1, team2):

    current_df = df[df['Year'] == '2023-24']

    #capturing the list of players in a variable
    team1_players = current_df[current_df['TEAM'] == team1]['PLAYER'].unique()
    team2_players = current_df[current_df['TEAM'] == team2]['PLAYER'].unique()
    
    performance1 = team_performance.team_analysis(current_df, team1)
    performance2 = team_performance.team_analysis(current_df, team2)
    
    words = [html.Div([html.P(f"{team1}'s team data: {performance1}"),
                        html.P(f"{team2}'s team data: {performance2}")
                            ])]

    # returning the options and analysis
    return [{'label': player, 'value': player} for player in team1_players], \
           [{'label': player, 'value': player} for player in team2_players], \
            words


#Callback to update button
@app.callback(
    Output('trade-button', 'children'),
    [Input('team1-players', 'value'),
     Input('team2-players', 'value')]
)

def button_label(team1_players, team2_players):
    if team1_players and team2_players:
        return f"Trade {len(team1_players)} players from Team 1 with {len(team2_players)} players from Team 2"
    else:
        return "Trade Selected Players"
    
#Callback to simulate trade
@app.callback(
    [Output('trade-analysis', 'children')],
    [Input('trade-button', 'n_clicks')],
     [State('team1-players', 'value'),
     State('team2-players', 'value'),
    State('team1-dropdown', 'value'),
     State('team2-dropdown', 'value')]
     )

def simulate_trade(n_clicks, team1_players, team2_players, team1, team2):
    if n_clicks and team1_players and team2_players:
        #Swap the teams for the selected players
        df.loc[df['PLAYER'].isin(team1_players), 'TEAM'] = team2
        df.loc[df['PLAYER'].isin(team2_players), 'TEAM'] = team1
        
        new_performance1 = team_performance.team_analysis(df, team1)
        new_performance2 = team_performance.team_analysis(df, team2)

        show = [html.Div([
            html.H3('Trade Analysis'),
            html.P(f"{team1}'s team data: {new_performance1}"),
            html.P(f"{team2}'s team data: {new_performance2}")
            ])]
        return show
    
    else:
        no_show = [html.Div([
        html.H3('Trade Analysis'),
        html.P('No trade executed yet.')
    ])]
    return no_show

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port = 8501)